In [ ]:
import pandas as pd
pd.set_option("display.max_columns",100)
import numpy as np

         # Clsassification Models
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeRegressor

          # Modelling
from sklearn.model_selection import train_test_split

          # Testing
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report



from imblearn.over_sampling import SMOTE


         # Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("train.csv")
d = {'Negado':0, 'Aprovado':1}
df["status_do_caso"]=df["status_do_caso"].map(d)
df_test = pd.read_csv("test.csv")
df = df.append(df_test)
d = {'Ensino Médio':80, 'Mestrado':160, 'Ensino Superior':120, 'Doutorado':200}
df["educacao_do_empregado"]=df["educacao_do_empregado"].map(d)
d = {"S":1,"N":0}
df["tem_experiencia_de_trabalho"]=df["tem_experiencia_de_trabalho"].map(d)
d={"S":1,"N":0}
df["requer_treinamento_de_trabalho"]=df["requer_treinamento_de_trabalho"].map(d)
df['ano_de_estabelecimento'] = 2023 - df['ano_de_estabelecimento']
df.loc[df['unidade_de_salario'] == 'Hora', 'salario_prevalecente'] *= 2080
df.loc[df['unidade_de_salario'] == 'Semana', 'salario_prevalecente'] *= 52
df.loc[df['unidade_de_salario'] == 'Mes', 'salario_prevalecente'] *= 12
df['salario_anual'] = df['salario_prevalecente']
df['posicao_em_tempo_integral'] = df['posicao_em_tempo_integral'].map({'S': 1, 'N': 0})
df['efeito_salario_emprego'] = df['salario_anual'] * df['posicao_em_tempo_integral']
df['qualificado_trabalho'] = df['educacao_do_empregado'] + df['tem_experiencia_de_trabalho']
df['empresa_grande'] = df['num_de_empregados'].apply(lambda x: 1 if x > 500 else 0)
df['salario_em_relacao_ao_num_de_empregados'] = df['salario_prevalecente'] / df['num_de_empregados']
df['trabalho_e_experiencia'] = df['requer_treinamento_de_trabalho'] + df['tem_experiencia_de_trabalho']
media_salario = df['salario_prevalecente'].mean()
df['salario_abaixo_media'] = df['salario_prevalecente'].apply(lambda x: 1 if x < media_salario else 0)
del df["id_do_caso"]
data = pd.get_dummies(df,drop_first=True)
train = data[:17836]
test = data[17836:]
x = train.drop("status_do_caso",axis=1)
y = train["status_do_caso"]
sm = SMOTE(random_state=42)
x_res, y_res = sm.fit_resample(x, y)
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2, random_state=42)
gradBoodClas = GradientBoostingClassifier()
model =gradBoodClas.fit(x_train,y_train)
guess = model.predict(test.drop("status_do_caso",axis=1))
result = pd.DataFrame()
result["id_do_caso"] =df_test["id_do_caso"]
result["status_do_caso"] = guess
result["status_do_caso"] = result["status_do_caso"].astype('int32')
result.to_csv("sample_submission_egitim_sayilari_artti.csv",index=False)